<a href="https://colab.research.google.com/github/SrivenkateswaraRao/Research-Project/blob/master/Pest_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from os import listdir
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import numpy as np

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import files
files.upload() 

In [4]:
import CreatePestImageData as Imagedataset
import SupportingFunctions as support

In [ ]:
support.DownLoadFiles('1aXb25Wp9sG7mL6AQgYY9ElrGcERi95JU',drive)

In [12]:
support.Unzip('/content/GREENPATROL_DB_V3.0.zip','/content/Pests/Images')

In [13]:
support.fileCount('/content/Pests/Images')

4330


In [14]:
dataSet = Imagedataset.createImageDataset()

In [16]:
class_object = {0 : 'egg_wf' ,
                1 : 'egg_bt', 2 : 'egg_ta', 3 : 'wf', 4 : 'bt', 5 : 'tomato'}

In [ ]:
dataSet.prepare_dataset('/content/Pests/Labels' , class_object ,32, '/content/Pests/Images/',  test_size=11241 )

In [ ]:
listdir('/content/Pests/Labels')

In [19]:
(trainY , trainX) , (testY , testX ) = (dataSet.trainY , dataSet.trainX ) , (dataSet.testY , dataSet.testX )


In [ ]:
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
import tensorflow as tf

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [20]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(44964, 32, 32, 3)
(44964,)
(11241, 32, 32, 3)
(11241,)


In [2]:
import numpy as np 
trainX = np.load('/content/drive/My Drive/Research-Data/trainX.npy') 
trainY = np.load('/content/drive/My Drive/Research-Data/trainY.npy') 
testX = np.load('/content/drive/My Drive/Research-Data/testX.npy') 
testY = np.load('/content/drive/My Drive/Research-Data/testY.npy') 
testX_Pred = np.load('/content/drive/My Drive/Research-Data/testX_Pred.npy') 
testY_Pred = np.load('/content/drive/My Drive/Research-Data/testY_Pred.npy')


In [23]:
import numpy as np 
np.save('/content/drive/My Drive/Research-Data/trainX',trainX) 
np.save('/content/drive/My Drive/Research-Data/trainY',trainY) 
np.save('/content/drive/My Drive/Research-Data/testX',testX) 
np.save('/content/drive/My Drive/Research-Data/testY', testY) 

In [27]:
import tensorflow as tf
from keras import models
from keras.applications.vgg16 import VGG16
vgg = VGG16(input_shape=[32,32,3], include_top=False, pooling='avg',weights='imagenet')
vgg.summary()

58892288/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
______________________________________________________________

In [28]:
from keras.models import Model
layerName = 'block1_pool'
mymodel = Model(inputs= vgg.input , outputs=vgg.get_layer(layerName).output)
mymodel.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
Total params: 38,720
Trainable params: 38,720
Non-trainable params: 0
_________________________________________________________________


In [29]:
from keras import applications
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization

In [30]:
model= models.Sequential()
model.add(mymodel)
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(GlobalAveragePooling2D())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(6, activation='softmax'))
model.layers[0].trainable = False

# compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [33]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 16, 16, 64)        38720     
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 16)        9232      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 16)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 32)          4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
global_average_pooling2d_2 ( (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)               

In [24]:
from tensorflow.keras.utils import to_categorical
trainY = to_categorical(trainY)
testY = to_categorical(testY)

In [25]:
filepath="/content/drive/My Drive/Models/VGG16/weights-{epoch:02d}-{accuracy:.3f}.hdf5"
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath, monitor='accuracy',
                               verbose=1, mode='max',save_best_only=True)

callbacks_list = [checkpoint]

Using TensorFlow backend.


In [32]:
tf.config.experimental_run_functions_eagerly(True)


In [ ]:
print(model.history.history.keys())

dict_keys(['val_loss', 'val_accuracy', 'loss', 'accuracy'])


In [34]:
model.fit(trainX,trainY,verbose=1,epochs=50,validation_data=(testX,testY),callbacks=[checkpoint])

Train on 44964 samples, validate on 11241 samples
Epoch 1/50
44964/44964 [==============================] - 29s 634us/step - loss: 1.0087 - accuracy: 0.6087 - val_loss: 0.9850 - val_accuracy: 0.5454

Epoch 00001: accuracy improved from -inf to 0.60873, saving model to /content/drive/My Drive/Models/VGG16/weights-01-0.609.hdf5
Epoch 2/50
44964/44964 [==============================] - 28s 627us/step - loss: 0.9036 - accuracy: 0.6386 - val_loss: 1.1773 - val_accuracy: 0.5181

Epoch 00002: accuracy improved from 0.60873 to 0.63856, saving model to /content/drive/My Drive/Models/VGG16/weights-02-0.639.hdf5
Epoch 3/50
44964/44964 [==============================] - 28s 625us/step - loss: 0.8634 - accuracy: 0.6530 - val_loss: 1.5526 - val_accuracy: 0.6471

Epoch 00003: accuracy improved from 0.63856 to 0.65303, saving model to /content/drive/My Drive/Models/VGG16/weights-03-0.653.hdf5
Epoch 4/50
44964/44964 [==============================] - 27s 608us/step - loss: 0.8633 - accuracy: 0.6581 - v

In [42]:
import json
history_dict = model.history.history
json.dump(str(history_dict), open('/content/drive/My Drive/Models/VGG16/History/VGG16_History4.json', 'w'))

In [36]:
(augtrainX,augtrainY) = support.PrepareAugmentedData(trainX,trainY)

In [44]:

from keras.models import load_model
model = load_model("/content/drive/My Drive/Models/VGG16/weights-25-0.767.hdf5")
model.fit(trainX,trainY,verbose=1,epochs=25,validation_data=(testX,testY),callbacks=[checkpoint])

Train on 44964 samples, validate on 11241 samples
Epoch 1/25
44964/44964 [==============================] - 45s 1ms/step - loss: 0.5693 - accuracy: 0.7680 - val_loss: 0.8500 - val_accuracy: 0.6870

Epoch 00001: accuracy improved from 0.76692 to 0.76797, saving model to /content/drive/My Drive/Models/VGG16/weights-01-0.768.hdf5
Epoch 2/25
44964/44964 [==============================] - 43s 954us/step - loss: 0.5696 - accuracy: 0.7677 - val_loss: 0.8212 - val_accuracy: 0.6998

Epoch 00002: accuracy did not improve from 0.76797
Epoch 3/25
44964/44964 [==============================] - 43s 949us/step - loss: 0.5668 - accuracy: 0.7688 - val_loss: 0.8354 - val_accuracy: 0.6939

Epoch 00003: accuracy improved from 0.76797 to 0.76877, saving model to /content/drive/My Drive/Models/VGG16/weights-03-0.769.hdf5
Epoch 4/25
44964/44964 [==============================] - 42s 931us/step - loss: 0.6165 - accuracy: 0.7503 - val_loss: 0.7986 - val_accuracy: 0.6939

Epoch 00004: accuracy did not improve f